In [199]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [200]:
# Load the data
df = pd.read_csv("businesses.csv", encoding="utf-8")
df_2 = pd.read_csv("production.csv", encoding="utf-8")
#Add df_2 to df
df.head()


,id,name,latitude,longitude,country,state,city,district,neighbourhood,suburb,street,postcode,categories
0,6009880430,ITV A-42,40.333884,-3.722897,Spain,Community of Madrid,Madrid,NaN,NaN,Villaverde,Autovía de Toledo,28021,"['commercial', 'service', 'service.vehicle', '..."
1,5016125471,El Almacen del Pino,40.333118,-3.713187,Spain,Community of Madrid,Madrid,NaN,NaN,Villaverde,Avenida Real de Pinto,28091,"['commercial', 'commercial.furniture_and_inter..."
2,5488430110,Casa,40.333391,-3.699976,Spain,Community of Madrid,Madrid,NaN,NaN,Villaverde,Calle Intermedia Sur,28021,"['building', 'building.commercial', 'building...."
3,3158349531,Talleres Mialdea,40.333948,-3.715134,Spain,Community of Madrid,Madrid,NaN,NaN,Villaverde,Calle del Valle de Tobalina,28091,"['commercial', 'service', 'service.vehicle', '..."
4,3295044255,Mercadona,40.334024,-3.711849,Spain,Community of Madrid,Madrid,NaN,NaN,Villaverde,Calle Laguna del Marquesado,28091,"['commercial', 'commercial.supermarket']"


In [201]:
# Split the categories into a list, remove brackets from the strings that have them and the single quotation marks too
df["categories"] = df["categories"].str.replace("[", "").str.replace("]", "").str.replace("'","").str.split(", ")
df_cats = df.explode("categories")
print(df_cats.categories)

0                       commercial
0                          service
0                  service.vehicle
0           service.vehicle.repair
0       service.vehicle.repair.car
                   ...            
9791    service.beauty.hairdresser
9792                    commercial
9792              commercial.books
9793                    commercial
9793              commercial.books
Name: categories, Length: 28866, dtype: object


In [202]:
#Return a list of all the unique categories
print(df_cats.categories.unique())

categories = df_cats.categories.unique()

['commercial' 'service' 'service.vehicle' 'service.vehicle.repair'
 'service.vehicle.repair.car' 'commercial.furniture_and_interior'
 'wheelchair' 'wheelchair.limited' 'building' 'building.commercial'
 'building.residential' 'commercial.supermarket' 'rental' 'rental.storage'
 'commercial.food_and_drink' 'commercial.food_and_drink.butcher'
 'building.industrial' 'commercial.outdoor_and_sport' 'commercial.vehicle'
 'service.beauty' 'service.beauty.hairdresser' 'commercial.convenience'
 'commercial.food_and_drink.confectionery' 'wheelchair.yes'
 'commercial.discount_store'
 'commercial.food_and_drink.fruit_and_vegetable' 'commercial.toy_and_game'
 'commercial.furniture_and_interior.curtain'
 'commercial.houseware_and_hardware'
 'commercial.houseware_and_hardware.doityourself' 'commercial.hobby'
 'commercial.hobby.art' 'commercial.clothing'
 'commercial.clothing.clothes' 'commercial.health_and_beauty'
 'service.beauty.spa' 'commercial.pet' 'pet' 'pet.shop'
 'commercial.elektronics' 'commer

In [203]:
df_cats

,id,name,latitude,longitude,country,state,city,district,neighbourhood,suburb,street,postcode,categories
0,6009880430,ITV A-42,40.333884,-3.722897,Spain,Community of Madrid,Madrid,NaN,NaN,Villaverde,Autovía de Toledo,28021,commercial
0,6009880430,ITV A-42,40.333884,-3.722897,Spain,Community of Madrid,Madrid,NaN,NaN,Villaverde,Autovía de Toledo,28021,service
0,6009880430,ITV A-42,40.333884,-3.722897,Spain,Community of Madrid,Madrid,NaN,NaN,Villaverde,Autovía de Toledo,28021,service.vehicle
0,6009880430,ITV A-42,40.333884,-3.722897,Spain,Community of Madrid,Madrid,NaN,NaN,Villaverde,Autovía de Toledo,28021,service.vehicle.repair
0,6009880430,ITV A-42,40.333884,-3.722897,Spain,Community of Madrid,Madrid,NaN,NaN,Villaverde,Autovía de Toledo,28021,service.vehicle.repair.car
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9791,11318423623,Peluquería Ébano,40.520727,-3.776666,Spain,Community of Madrid,Madrid,Fuencarral-El Pardo,NaN,El Pardo,Calle Manuel Alonso,28048,service.beauty.hairdresser
9792,1544594083,Librería de la UAM (Psicología),40.543892,-3.692088,Spain,Community of Madrid,Madrid,Fuencarral-El Pardo,El Goloso,NaN,Calle Iván Pavlov,28049,commercial
9792,1544594083,Librería de la UAM (Psicología),40.543892,-3.692088,Spain,Community of Madrid,Madrid,Fuencarral-El Pardo,El Goloso,NaN,Calle Iván Pavlov,28049,commercial.books
9793,1544592697,Librería OMM,40.545471,-3.693443,Spain,Community of Madrid,Madrid,Fuencarral-El Pardo,El Goloso,NaN,Calle Francisco Tomás y Valiente,28049,commercial


In [204]:
#Create a dictionary with all the main categories (before the dot) as keys and a list of all the subcategories as values
categories_dict = {}
for cat in categories:
    main_cat = cat.split(".")[0]
    #Throw out the categories that have more than one dot
    if len(cat.split(".")) > 2:
        continue
    if main_cat in categories_dict:
        categories_dict[main_cat].append(cat)
    else:
        categories_dict[main_cat] = [cat]

In [211]:
new_cats_dict = {k: v for k, v in categories_dict.items() if k in ["commercial", "production", "office", "service", "catering"]}
#print the number of values of all the values as keys
print({k: len(v) for k, v in new_cats_dict.items()})
new_cats_dict["catering"]

{'commercial': 42, 'service': 12, 'catering': 8, 'office': 8, 'production': 2}


['catering',
 'catering.cafe',
 'catering.ice_cream',
 'catering.pub',
 'catering.fast_food',
 'catering.restaurant',
 'catering.taproom',
 'catering.bar']